In [1]:
import heapq
import math
import os
import re
import time
from typing import List, Optional, Tuple

import aocd
import numpy as np
from IPython.display import HTML, clear_output, display
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2024, day=16)

In [3]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[1].input_data
    else:
        data = p.input_data
    return data

In [4]:
def process_data(data):
    grid = [list(l) for l in data.split("\n")]
    grid = np.array(grid)
    return grid

In [5]:
print(get_data(test_data=True))

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################


In [6]:
data = get_data(test_data=False)
grid = process_data(data)

In [7]:
Coord = Tuple[int, int]


class MazePathFinder:
    """
    Find paths in a grid maze, minimizing:
        cost = steps + turn_cost * turns

    grid: 2D numpy array of single-char strings:
          '#', '.', 'S', 'E'
    """

    # Moves: (dr, dc, direction_name)
    DIRS = [
        (-1, 0, "U"),
        (1, 0, "D"),
        (0, -1, "L"),
        (0, 1, "R"),
    ]

    def __init__(self, grid: np.ndarray, turn_cost: int = 1000):
        self.grid = grid
        self.turn_cost = float(turn_cost)

        self.rows, self.cols = grid.shape

        # Parse start/end
        ((sr, sc),) = np.argwhere(grid == "S")
        ((er, ec),) = np.argwhere(grid == "E")
        self.start: Coord = (int(sr), int(sc))
        self.end: Coord = (int(er), int(ec))

        # Precompute walkable cells
        self.walkable = grid != "#"

        # Internal storage for Dijkstra results
        self._dist = None  # shape (rows, cols, 4)
        self._parents = None  # shape (rows, cols, 4)
        self._best_cost = None
        self._end_states = None

    # ---------- Public API ----------

    def find_optimal_paths(self, max_paths: Optional[int] = None) -> List[List[Coord]]:
        """
        Run Dijkstra (if not already run) and return all optimal paths
        (minimizing steps + turn_cost * turns).

        max_paths: if not None, stop after collecting this many paths.
        """
        if self._dist is None:
            self._run_dijkstra()

        if not self._end_states:
            return []

        self.all_paths = self._backtrack_all_paths(max_paths=max_paths)
        return self.all_paths

    def best_cost(self) -> Optional[float]:
        """Return best (minimal) cost found, or None if no path."""
        if self._dist is None:
            self._run_dijkstra()
        return self._best_cost

    @staticmethod
    def path_to_directions(path: List[Coord]) -> List[str]:
        """
        Convert a coordinate path to a list of 'U','D','L','R' moves.
        """
        dirs: List[str] = []
        for (r1, c1), (r2, c2) in zip(path, path[1:]):
            dr, dc = r2 - r1, c2 - c1
            if dr == 1 and dc == 0:
                dirs.append("D")
            elif dr == -1 and dc == 0:
                dirs.append("U")
            elif dr == 0 and dc == 1:
                dirs.append("R")
            elif dr == 0 and dc == -1:
                dirs.append("L")
            else:
                raise ValueError(f"Non-adjacent move: {(r1, c1)} -> {(r2, c2)}")
        return dirs

    def count_steps_and_turns(self, path: List[Coord]) -> Tuple[int, int, List[str]]:
        """
        For a given path, return (steps, turns, directions).
        cost = steps + turn_cost * turns
        """
        dirs = self.path_to_directions(path)
        steps = len(dirs)
        turns = sum(1 for a, b in zip(dirs, dirs[1:]) if a != b)
        return steps, turns, dirs

    def path_cost(self, path: List[Coord]) -> float:
        """Compute cost = steps + turn_cost * turns for a given path."""
        steps, turns, _ = self.count_steps_and_turns(path)
        return steps + self.turn_cost * turns

    def show_best_path(self):
        return f"Found {len(self.all_paths)} optimal paths with cost={self._best_cost:.0f}"

    def get_cells_in_best_paths(self) -> set[Coord]:
        """Return set of all cells that appear in any optimal path."""
        all_cells = set()
        for path in self.all_paths:
            for cell in path:
                all_cells.add(cell)
        return all_cells

    # ---------- Internal methods ----------

    def _run_dijkstra(self) -> None:
        """
        Dijkstra on (r, c, dir_index) states.

        - First step from S has no turn penalty.
        - dist[r, c, d] = best cost to reach (r,c) arriving with direction d.
        - parents[r, c, d] = list of (prev_r, prev_c, prev_d or None)
        """
        rows, cols = self.rows, self.cols
        walkable = self.walkable
        DIRS = self.DIRS
        DIR_COUNT = len(DIRS)
        TURN_COST = self.turn_cost
        (start_r, start_c) = self.start
        (end_r, end_c) = self.end

        dist = np.full((rows, cols, DIR_COUNT), math.inf, dtype=float)
        parents = np.empty((rows, cols, DIR_COUNT), dtype=object)
        parents[:] = None

        pq: List[Tuple[float, int, int, int]] = []  # (cost, r, c, dir_index)

        # Initialize: take first step out of S (no turn cost)
        for d_idx, (dr, dc, _) in enumerate(DIRS):
            nr, nc = start_r + dr, start_c + dc
            if 0 <= nr < rows and 0 <= nc < cols and walkable[nr, nc]:
                dist[nr, nc, d_idx] = 1.0
                parents[nr, nc, d_idx] = [(start_r, start_c, None)]
                heapq.heappush(pq, (1.0, nr, nc, d_idx))

        # Dijkstra with multi-parents
        while pq:
            cost, r, c, d_idx = heapq.heappop(pq)
            if cost != dist[r, c, d_idx]:
                continue

            for nd_idx, (dr, dc, _) in enumerate(DIRS):
                nr, nc = r + dr, c + dc
                if not (0 <= nr < rows and 0 <= nc < cols):
                    continue
                if not walkable[nr, nc]:
                    continue

                step_cost = 1.0
                turn_penalty = 0.0 if nd_idx == d_idx else TURN_COST
                new_cost = cost + step_cost + turn_penalty

                cur_best = dist[nr, nc, nd_idx]
                if new_cost < cur_best:
                    # strictly better -> replace dist and parents
                    dist[nr, nc, nd_idx] = new_cost
                    parents[nr, nc, nd_idx] = [(r, c, d_idx)]
                    heapq.heappush(pq, (new_cost, nr, nc, nd_idx))
                elif new_cost == cur_best:
                    # equally good -> record additional parent
                    if parents[nr, nc, nd_idx] is None:
                        parents[nr, nc, nd_idx] = []
                    parents[nr, nc, nd_idx].append((r, c, d_idx))

        # Store
        self._dist = dist
        self._parents = parents

        # Find best cost and corresponding end states
        best_cost = math.inf
        end_states = []
        for d_idx in range(DIR_COUNT):
            cst = dist[end_r, end_c, d_idx]
            if cst < best_cost:
                best_cost = cst
                end_states = [(end_r, end_c, d_idx)]
            elif cst == best_cost:
                end_states.append((end_r, end_c, d_idx))

        self._best_cost = best_cost if not math.isinf(best_cost) else None
        self._end_states = (
            end_states if end_states and not math.isinf(best_cost) else []
        )

    def _backtrack_all_paths(self, max_paths: Optional[int]) -> List[List[Coord]]:
        """
        Backtrack through parents to recover all optimal paths.
        Assumes _run_dijkstra has been called and _end_states is set.
        """
        if not self._end_states:
            return []

        parents = self._parents
        start = self.start
        all_paths: List[List[Coord]] = []

        # stack: (r, c, dir_index, path_backwards_list)
        stack: List[Tuple[int, int, Optional[int], List[Coord]]] = []
        for er, ec, d_idx in self._end_states:
            stack.append((er, ec, d_idx, [(er, ec)]))

        while stack:
            r, c, d_idx, path_back = stack.pop()

            parent_list = parents[r, c, d_idx]
            if parent_list is None:
                continue

            for pr, pc, pdir in parent_list:
                new_path_back = path_back + [(pr, pc)]
                if pdir is None:
                    # We've reached the start node
                    if (pr, pc) != start:
                        # sanity check – we expect this to be the start
                        continue
                    full_path = list(reversed(new_path_back))
                    all_paths.append(full_path)
                    if max_paths is not None and len(all_paths) >= max_paths:
                        stack = []
                        break
                else:
                    stack.append((pr, pc, pdir, new_path_back))

        # Deduplicate coordinate-paths (different dir-states can give same path)
        unique_paths: List[List[Coord]] = []
        seen = set()
        for p in all_paths:
            key = tuple(p)
            if key not in seen:
                seen.add(key)
                unique_paths.append(p)

        return unique_paths

## Part 1 - Dijkstra

In [8]:
%%time
finder = MazePathFinder(grid, turn_cost=1000)

paths = finder.find_optimal_paths(max_paths=50)

finder.show_best_path()

CPU times: user 33.5 ms, sys: 1.51 ms, total: 35 ms
Wall time: 34.8 ms


'Found 13 optimal paths with cost=85480'

## Part 2 

In [9]:
%%time
len(finder.get_cells_in_best_paths())

CPU times: user 162 μs, sys: 8 μs, total: 170 μs
Wall time: 172 μs


518

## Illustration

In [10]:
def html_grid(grid):
    colors = {
        "#": ("█", "#8d6e63"),  # Warm brown walls
        ".": ("·", "#4a4a4a"),  # Subtle grey dots
        "O": ("▣", "#f44336"),
        "@": ("◉", "#4caf50"),
    }
    background = "#2d2d2d"

    cells = ""
    for row in grid:
        for c in row:
            char, color = colors.get(c, (c, "#333333"))
            cells += f'<span style="color:{color};">{char}</span>'
        cells += "<br>"

    html = f"""
    <div style="font-family:monospace;font-size:16px;line-height:1.1;background:{background};padding:15px;border-radius:10px;display:inline-block;">
    {cells}
    </div>
    """
    return html

In [11]:
for pos in finder.get_cells_in_best_paths():
    y, x = pos
    grid[y, x] = "@"

clear_output(wait=True)
display(HTML(html_grid(grid)))